In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "2"
os.environ["CUDA_MODULE_LOADING"]= "LAZY"
import sys
sys.path.append('third_party/Matcha-TTS')

from cosyvoice.cli.cosyvoice import CosyVoice
from cosyvoice.utils.file_utils import load_wav
import torchaudio

cosyvoice = CosyVoice('./pretrained_models/CosyVoice-300M-son-ploonet',load_jit=False, load_trt=False)

import torch
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
from cosyvoice.utils.train_utils import (
init_distributed,
init_dataset_and_dataloader,
init_optimizer_and_scheduler,
init_summarywriter, save_model,
wrap_cuda_model, check_modify_and_save_config)
from hyperpyyaml import load_hyperpyyaml

config_file = "./pretrained_models/CosyVoice-300M-son-ploonet/cosyvoice.yaml"

override_dict = {k: None for k in ['llm', 'flow', 'hift'] if k != "llm"}

with open(config_file, 'r') as f:
    configs = load_hyperpyyaml(f, overrides=override_dict)

#configs['train_conf'].update(vars(args))

print(configs)

/data/minju/conda/envs/cosyvoice/lib/python3.10/site-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2025-02-07 10:52:11,714 INFO input frame rate=50
/data/minju/conda/envs/cosyvoice/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
festival_initialize() called more than once


text.cc: festival_Text_init
open voice lang map failed
{'sample_rate': 22050, 'text_encoder_input_size': 512, 'llm_input_size': 1024, 'llm_output_size': 1024, 'spk_embed_dim': 192, 'llm': TransformerLM(
  (text_embedding): Embedding(51866, 512)
  (text_encoder): ConformerEncoder(
    (embed): LinearNoSubsampling(
      (out): Sequential(
        (0): Linear(in_features=512, out_features=1024, bias=True)
        (1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (2): Dropout(p=0.1, inplace=False)
      )
      (pos_enc): EspnetRelPositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (after_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (encoders): ModuleList(
      (0-5): 6 x ConformerEncoderLayer(
        (self_attn): RelPositionMultiHeadedAttention(
          (linear_q): Linear(in_features=1024, out_features=1024, bias=True)
          (linear_k): Linear(in_features=1024, out_features=1024, bias=True)
          (linea

In [7]:
## 임베딩 파일 경로 변경
print(cosyvoice.list_available_spks())
print(len(cosyvoice.list_available_spks()))

emb_path = "/data/minju/TTS/CosyVoice/embedding_models/son-ploonet/spk2info.pt"
emb_path = "/data/minju/TTS/CosyVoice/embedding_models/slx/spk2info.pt"
# emb_path = "/data/minju/TTS/CosyVoice/embedding_models/semi/spk2info.pt"
emb_path = "/data/minju/TTS/CosyVoice/embedding_models/senti-slx/spk2info.pt"
# emb_path = "/data/minju/TTS/CosyVoice/embedding_models/cardergarden/spk2info.pt"
emb_path = "/data/minju/TTS/CosyVoice/embedding_models/emo-all-slx/spk2info.pt"
emb_path = "/data/minju/TTS/CosyVoice/embedding_models/multi/spk2info.pt"
emb_type = emb_path.split("/")[-2]
spk2info = torch.load(emb_path, map_location = "cuda")
cosyvoice.frontend.spk2info = spk2info

print(cosyvoice.list_available_spks())
print(len(cosyvoice.list_available_spks()))

['tony', 'elley', 'roy', 'eden', 'rudy', 'noa', 'ruby', 'lye', 'jenny', 'secretary_ian', 'secretary_roun', 'secretary_joy', 'bitna_bright', 'bitna_default', 'hayun_bright_1.05', 'hayun_default_1.05', 'kyeon_bright', 'kyeon_default', 'sanggyun_default', 'sehyeok_bright', 'sehyeok_default', 'wansun_default_1.05', 'yongmin_default_1.05', 'yuna_bright', 'yuna_default_1.05', 'boram_bright', 'tony_default_1.05', 'boram_default', 'kyungil_default_1.05', 'slx_speakers_KAIST_males_2', 'slx_speakers_del_JIHYANG', 'slx_speakers_SEHA_fastpitch', 'donghee_default', 'donghee_bright']
34
['tony', 'elley', 'roy', 'eden', 'rudy', 'noa', 'ruby', 'lye', 'jenny', 'secretary_ian', 'secretary_roun', 'secretary_joy', 'bitna_bright', 'bitna_default', 'hayun_bright_1.05', 'hayun_default_1.05', 'kyeon_bright', 'kyeon_default', 'sanggyun_default', 'sehyeok_bright', 'sehyeok_default', 'wansun_default_1.05', 'yongmin_default_1.05', 'yuna_bright', 'yuna_default_1.05', 'boram_bright', 'tony_default_1.05', 'boram_def

In [8]:
from IPython.display import Audio
import os

In [11]:
# sft usage
# change stream=True for chunk stream inference
epoch = 15

model = configs["llm"]
model_type = "cosyvoice" #semi
model_type = "250110-slx"
model_type = "250114-senti-slx"
# model_type = "250111-son-ploonet"
model_type = "250115-emo-all-slx"
model_type = "250203-multi"
model.load_state_dict(torch.load(f"/data/minju/TTS/CosyVoice/examples/libritts/cosyvoice/exp/{model_type}/llm/torch_ddp/epoch_{epoch}_whole.pt", map_location='cuda'), strict=False)
cosyvoice.model.llm = model.to("cuda")

speaker = "yuna_default_1.05"
speaker = "secretary_joy"
# speaker = "sunbeacon"
# speaker = "0988_G1A3E7_JSW"
text = "오늘부터 사나흘에 걸쳐 충남과 호남 지역 제주도 등에 많은 눈이 내리겠습니다. 이 시각 대설주의보가 내려진 전북 고창의 모습을 보면 거리에 눈이 쌓인 가운데 굵은 눈이 펑펑 쏟아지고 있습니다. 서해안에서 강한 눈구름대가 유입되면서 서쪽 지역을 중심으로 눈이 내리고 있습니다."
# text = "이 시각 대설주의보가 내려진 전북 고창의 모습을 보면 거리에 눈이 쌓인 가운데 굵은 눈이 펑펑 쏟아지고 있습니다."
text = "끊임없이 올라가기만 하고 재미없어요. 내려오기도 해야 재밌죠. 오르락 내리락이있으니까 재미라는게 생기는거죠."
text = "여당은 헌재가 권한쟁의심판에서 국회 측 청구를 인용해 마 후보자를 임명해야 한다는 결론을 내릴 가능성이 높다는 전망이 나오자 이런 주장을 펴기 시작했다."
text = "그룹 블랙핑크 로제와 팝스타 브루노 마스가 함께 부른 아파트가 빌보드 차트에서 전주보다 이십 구 계단 상승하며 오위에 올랐습니다."
#text = "이 직무대행은 13일 국회 행정안전위원회 현안 질의에 출석해 더불어민주당 모경종 의원으로부터 국민의힘 의원들의 체포 저지 행동에 대한 질문을 받고 적극적으로 체포를 저지하면 현행범이 될 수 있다고 말했다."
# text = "대장 내시경 검사가 대장암 발생률과 사망률을 낮추는 데 큰 효과가 있다는 대규모 분석 결과가 나왔어."
# text = "그리운 연진에게 혹시 기억하니? 내가 여름을 아주 싫어했던거 다행히 더 더워지기 전에 이사를 했어."
# text = "공식적으로 대통령 신분이 된 트럼프 당선인은 상원 의사당 근처에 있는 대통령실로 향하게 된다. 그곳에서 지명자 서명식을 갖는 것이 천구백팔십일년 로널드 레이건 전 대통령 때부터 이어져 온 전통이다. 이후엔 의회 합동위원회 오찬 군 사열 및 펜실베이니아 에비뉴 퍼레이드, 백악관 집무실 서명 행사 세 차례 무도회 등의 순으로 행사가 진행된다."
# text = "국립민속박물관이 발간한 한국민속예술사전 한국학중앙연구원의 한국민족문화대백과사전 등에 따르면 공기는 조선시대 화가 윤덕희의 공기놀이 그림이나 헌종 때 실학자 이규경의 오주연문장. 오주연문장전산고엔 우리나라 아이들이 둥근 돌알을 가지고 노는 놀이가 있어 공기라고 한다라는 기록이 있다. 고려대 민족문화연구소의 한국민속대관에 따르면 오늘날 공기놀이라는 명칭이 일반화됐으나."
#text = "국립민속박물관이 발간한 한국민속예술사전 한국학중앙연구원의 한국민족문화대백과사전 등에 따르면 공기는 조선시대 화가 윤덕희의 공기놀이 그림이나 헌종 때 실학자 이규경의 오주연문장전산고에 등장할 만큼 그 유래가 오래됐다. 오주연문장전산고엔 우리나라 아이들이 둥근 돌알을 가지고 노는 놀이가 있어 공기라고 한다라는 기록이 있다. 고려대 민족문화연구소의 한국민속대관에 따르면 오늘날 공기놀이라는 명칭이 일반화됐으나 지방마다 다른 이름으로 불리기도 했다."
# text = "출장까지 와서 뭔 아침에 조깅을 했다질 않나 꽃파는 할머니한테 꽃다발을 사왔다지 않나. 그건 또 무슨 코스프레야."
# text = "김 직무대행은 국민 불편을 최소화하도록 철저히 준비하겠다며 이진숙 위원장에 대한 탄핵 심판을 진행 중인 헌법재판소와 국회의 협조로 하루빨리 기관이 정상화되길 희망한다고 말했습니다."
#text = "공식적으로 대통령 신분이 된 트럼프 당선인은 상원 의사당 근처에 있는 대통령실로 향하게 된다. 그곳에서 지명자 서명식을 갖는 것이 천구백팔십일년 로널드 레이건 전 대통령 때부터 이어져 온 전통이다."
#text = "선수 시절뿐 아니라 각종 선거에서 이변을 일으켜온 그답게 이번에도 그 진가를 보여줬다. 이번 체육회장 선거는 이기흥 현 회장 당선 가능성이 높다는 예상이 지배적이었다. 선거인단 구성 자체도 이 회장에게 유리하다는 지적이 있었다. 추첨으로 뽑히는 인원 외에 전국 이백이십팔개 시 군 구 체육회에서 추천한 인사가 선거인단에 포함되는 지정 선거인 제도가 도입됐는데 현직 회장으로 지역 체육회와 접촉이 많았던 이 회장을 위한 제도라는 비판이 있었다. 실제 이날 투표 현장에선 유 후보가 당선되자 환호성을 지르는 젊은 체육인들이 대거 눈에 띄었다. 대한체육회 전직 임원은 젊은 체육인들이 변화를 원했다. 체육인들을 만나보면 한국 체육계가 나이 드신 분들이 권위를 내세워 끌고 간다는 비판이 많았다. 젊은 사람이 와서 변화를 이끌어줬으면 하는 바람이 많았다고 전했다."
text = "그날 밤 삼신할매는 다녀가지 않았고 어머니는 여전히 반대하신다. 그래도 우리는 사랑을 하고 있다. 투닥투닥 싸우고 화해하고 웃고 울고 연애질을 한다. 가끔은 그런 생각을 한다. 어쩌면 우리도 헤어질 수 있겠구나. 연애라는 게 그런 거니까. 하지만 미리 두려워하지는 않겠다. 지금 내가 해야 할 일은 명백하다. 열심히 케이크를 굽고, 열심히 살아가는 것. 오늘이 마지막인 것처럼. 한번도 상처받지 않은 것처럼. 나 김삼순을 더 사랑하는 것."
# text = "맞아요. 그래서 제가 조사한 자료를 보시면 이십대가 주로 사용하는 플랫폼은 인스타그램과 틱톡이에요. 이쪽에 초점을 맞춰야 할 것 같아요."
# text = "다들 이번 주말에 어디 여행 갈래? 날씨도 좋은데 가까운 데로 당일치기 여행 어때?"
# text = "선수 시절뿐 아니라 각종 선거에서 이변을 일으켜온 그답게 이번에도 그 진가를 보여줬다. 이번 체육회장 선거는 이기흥 현 회장 당선 가능성이 높다는 예상이 지배적이었다. 선거인단 구성 자체도 이 회장에게 유리하다는 지적이 있었다. 추첨으로 뽑히는 인원 외에 전국 이백이십팔개 시 군 구 체육회에서 추천한 인사가 선거인단에 포함되는 지정 선거인 제도가 도입됐는데 현직 회장으로 지역 체육회와 접촉이 많았던 이 회장을 위한 제도라는 비판이 있었다. 실제 이날 투표 현장에선 유 후보가 당선되자 환호성을 지르는 젊은 체육인들이 대거 눈에 띄었다. 대한체육회 전직 임원은 젊은 체육인들이 변화를 원했다. 체육인들을 만나보면 한국 체육계가 나이 드신 분들이 권위를 내세워 끌고 간다는 비판이 많았다. 젊은 사람이 와서 변화를 이끌어줬으면 하는 바람이 많았다고 전했다."
text = "음... 우리가 매일 사용하는 에이아이 서비스들이 더욱 빨라지고 정교해질 거예요. 자율주행차, 의료 진단, 기후 예측 등 고성능 컴퓨팅이 필요한 서비스들이 더욱 발전하게 될 거고요."
# text = "여러분 칠조원이라는 돈이 얼마나 큰 금액인지 아시나요? 이는 서울시의 일년 예산과 맞먹는 규모인데 오늘은 글로벌 투자회사가 단 하나의 데이터센터 기업에 투자하기로 한 놀라운 금액입니다. 정말 어마어마한 규모네요. 맥쿼리가 애플라이드 디지털에 투자하기로 한 오십억 달러라는 금액이 주는 충격이 상당한데요. 그렇죠. 특히 흥미로운 점은 이 투자가 단순한 돈의 흐름이 아니라 에이아이 시대로의 본격적인 전환을 알리는 신호라는 거예요. 시장 조사기관들은 향후 오년간 에이아이와 에이치피씨 관련 데이터센터 투자가 매년 이십 퍼센트 이상 성장할 것으로 전망하고 있거든요. 애플라이드 디지털이라는 회사가 꽤 흥미로운 이력을 가지고 있다고 들었는데요? 맞아요. 이 회사는 원래 암호화폐 채굴 하드웨어 호스팅으로 시작했다가 에이아이와 에이치피씨로 방향을 전환한 케이스예요. 현재는 노스다코타와 텍사스에서 삼백팔십 메가와트가 넘는 용량을 운영하고 있죠. 그런데 이런 대규모 투자에는 항상 위험도 따르기 마련인데 이번 투자의 리스크는 어떻게 보시나요? 가장 큰 도전 과제는 빠르게 변화하는 기술 환경에 대한 적응일 것 같아요. 특히 에이아이 기술은 매우 빠르게 발전하고 있어서 오늘의 최첨단 시설이 내일은 구식이 될 수 있거든요. 하지만 맥쿼리는 이런 리스크를 충분히 고려했을 것 같은데요. 특히 투자 구조를 보면 상당히 전략적으로 접근한 것으로 보입니다. 네 맞습니다. 우선 구억 달러를 즉시 투자하고 이후 삼십개월 동안 사십일억 달러를 추가로 투자할 수 있는 옵션을 확보했거든요. 이는 시장 상황을 지켜보면서 유연하게 대응할 수 있는 구조죠. 엘렌데일 캠퍼스 프로젝트가 특히 주목받고 있다고 하던데요. 그렇죠. 사백 메가와트 규모의 이 캠퍼스는 정말 인상적인 시설이 될 거예요. 백팔십 메가와트는 암호화폐 하드웨어를 백 메가와트는 에이티피 호스팅을 위해 설계됐는데 이는 미래의 수요 변화에도 유연하게 대응할 수 있는 구조입니다. 웨스 커민스 씨이오의 비전도 꽤나 야심찬 것 같아요. 맞아요. 그는 이번 투자로 이 기가와트 이상의 에이치피씨 데이터센터 용량을 구축할 수 있게 됐다고 밝혔어요. 이는 미국 내 가장 빠르게 성장하는 에이치피씨 데이터센터 운영자가 되겠다는 포부를 보여주는 거죠. 이런 변화가 일반 사람들의 삶에는 어떤 영향을 미칠까요? 우리가 매일 사용하는 에이아이 서비스들이 더욱 빨라지고 정교해질 거예요. 자율주행차 의료 진단 기후 예측 등 고성능 컴퓨팅이 필요한 서비스들이 더욱 발전하게 될 거고요. 결국 이번 투자는 에이아이 시대로의 본격적인 전환점이 될 것 같네요. 맞습니다. 칠조원이라는 거대한 투자금은 에이아이가 더 이상 먼 미래의 이야기가 아니라는 걸 보여주고 있어요. 이제 우리는 에이아이 혁명의 한가운데에 서 있는 거죠. 앞으로도 이 분야의 발전을 주의 깊게 지켜볼 필요가 있을 것 같습니다. 오늘도 흥미진진한 이야기 감사합니다. 네 다음 시간에도 새로운 이야기로 찾아뵙겠습니다. 청취해 주셔서 감사합니다."
# text = "선수 시절뿐 아니라 각종 선거에서 이변을 일으켜온 그답게 이번에도 그 진가를 보여줬다. 이번 체육회장 선거는 이기흥 현 회장 당선 가능성이 높다는 예상이 지배적이었다. 선거인단 구성 자체도 이 회장에게 유리하다는 지적이 있었다. 추첨으로 뽑히는 인원 외에 전국 이백이십팔개 시 군 구 체육회에서 추천한 인사가 선거인단에 포함되는 지정 선거인 제도가 도입됐는데 현직 회장으로 지역 체육회와 접촉이 많았던 이 회장을 위한 제도라는 비판이 있었다. 실제 이날 투표 현장에선 유 후보가 당선되자 환호성을 지르는 젊은 체육인들이 대거 눈에 띄었다. 대한체육회 전직 임원은 젊은 체육인들이 변화를 원했다. 체육인들을 만나보면 한국 체육계가 나이 드신 분들이 권위를 내세워 끌고 간다는 비판이 많았다. 젊은 사람이 와서 변화를 이끌어줬으면 하는 바람이 많았다고 전했다."
# text = '올 해는 다른 해 보다 카페도 많이 가고 지방 식당도 많이 갔습니다만 필사적으로 서울 식당들을 많이 돌아다닌 해이기도 했던 것 같습니다.'
output_path = "output/250115"
os.makedirs(output_path, exist_ok=True)
save_path = f"{output_path}/sft_{model_type}_{epoch}_{speaker}_{text[:3]}"

total_tensor = []
 
for i, j in enumerate(cosyvoice.inference_sft(text, speaker, stream=False)):
    print(j['tts_speech'])
    total_tensor.append(j['tts_speech'])
    torchaudio.save(f"{save_path}_{i}.wav", j['tts_speech'], cosyvoice.sample_rate)

concatenated_tensor = torch.cat(total_tensor, dim=-1)
torchaudio.save(f"{save_path}_{i+1}.wav", concatenated_tensor, cosyvoice.sample_rate)
Audio(f"{save_path}_{i+1}.wav")

2025-02-07 10:55:39,934 INFO synthesis text 음… 우리가 매일 사용하는 에이아이 서비스들이 더욱 빨라지고 정교해질 거예요。자율주행차，의료 진단，기후 예측 등 고성능 컴퓨팅이 필요한 서비스들이 더욱 발전하게 될 거고요。


update! 음… 우리가 매일 사용하는 에이아이 서비스들이 더욱 빨라지고 정교해질 거예요. 자율주행차，의료 진단，기후 예측 등 고성능 컴퓨팅이 필요한 서비스들이 더욱 발전하게 될 거고요. 
75


2025-02-07 10:56:12,721 INFO yield speech len 18.390204081632653, rtf 1.7828363634180278
100%|██████████| 1/1 [00:32<00:00, 32.81s/it]

tensor([[-0.0032, -0.0016,  0.0048,  ..., -0.0005, -0.0004, -0.0004]])
